## LangChain 経由で SageMaker でホストした大規模言語モデル (LLM) を使う

### このノートブックについて  

こちらは大規模言語モデル（LLM）を使ったアプリケーションを構築するためのライブラリーである [langchain](https://langchain.com/) を用いて SageMaker 上でホストした LLM から推論結果を得るサンプルコードを示したものです。  

LLM の例として今回は HuggingFace 上で rinna 社が公開している [rinna/japanese-gpt-neox-3.6b-instruction-ppo](https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-ppo) を使用する想定です。   
 

こちらの Notebook は 以下の環境で動作確認を行っています。

- SageMaker Studio Notebooks 
  - `ml.t3.medium`: `Data Science 3.0`
- SageMaker Notebooks
    - `ml.t3.medium`: `conda_pytorch_p310`

[各インスタンスの料金についてはこちら](https://aws.amazon.com/jp/sagemaker/pricing/)をご確認ください。  


また、ノートブックを動かすにあたって、各セルを上から順番に実行すれば動きますが、SageMaker 上での推論の仕組みについては、[AI/ML DarkPark](https://www.youtube.com/playlist?list=PLAOq15s3RbuL32mYUphPDoeWKUiEUhcug) の特に [Amazon SageMaker 推論 Part2すぐにプロダクション利用できる！モデルをデプロイして推論する方法 【ML-Dark-04】【AWS Black Belt】](https://youtu.be/sngNd79GpmE) をご参照ください。

### 前準備  

まずは事前準備として LLM を SageMaker Realtime Endpoint でホストします。  
下記リンクの Notebook を実行することでエンドポイントを立てます。    
https://github.com/aws-samples/aws-ml-jp/blob/main/tasks/generative-ai/text-to-text/fine-tuning/instruction-tuning/Transformers/Rinna_Neox_Inference_ja.ipynb


In [ ]:
# !wget https://raw.githubusercontent.com/aws-samples/aws-ml-jp/main/tasks/generative-ai/text-to-text/fine-tuning/instruction-tuning/Transformers/Rinna_Neox_Inference_ja.ipynb
# Notebook をコピーする必要がある場合はこちらのコメントアウトを外して実行しダウンロードした Notebook を実行します。

立ち上げたエンドポイント名を下の値に置き換えます。

In [ ]:
# endpoint_name = <エンドポイント名>
endpoint_name = 'Rinna-Inference'

### 必要モジュールのインストール

In [ ]:
!pip install 'langchain>=0.0.186'

## LangChain を使ってみる  

必要モジュールのインストールが完了したので、ここから実際に LangChain を使ってみましょう。

### 必要モジュールの import 

In [ ]:
import codecs
import json
from typing import Dict

from langchain.docstore.document import Document
from langchain import PromptTemplate, SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain

In [ ]:
region_name = "us-east-1"  # 適宜使っているリージョン名に書き換えてください

ここで LLM からのレスポンスから適切に文字列を抜き出すための操作を ContentHandler という名前のクラスで定義します。  

ここで受け付ける入力と出力の形式ははホストする LLM ごとによって変化しうるので注意してください。  

rinna では改行コードとして`<NL>`を使っているので置き換えも行なっています。


In [ ]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input_str = json.dumps(
            {
                "input": prompt.replace("\n", "<NL>"), 
                "instruction": "", 
                **model_kwargs
            })
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json.replace("<NL>", "\n")

LangChain では、LLM が応答の根拠として使用する文書を `Document` オブジェクトとして管理します。今回は簡易的な SageMaker に関する説明文を使って、質問に回答させてみます。

In [ ]:
example_doc_1 = """
Amazon SageMakerは、フルマネージド型の機械学習サービスです。SageMakerを利用することで、データサイエンティストや開発者は、機械学習モデルを迅速かつ容易に構築・訓練し、本番環境に直接デプロイすることができます。Jupyterオーサリングノートブックのインスタンスを統合して提供し、データソースに簡単にアクセスして探索や分析を行うことができるため、サーバーを管理する必要がありません。また、分散環境で非常に大きなデータに対して効率的に実行できるように最適化された、一般的な機械学習アルゴリズムも提供します。SageMakerは、Bring-your-own-algorithmsとフレームワークのネイティブサポートにより、特定のワークフローに適応する柔軟な分散トレーニングオプションを提供します。SageMaker StudioまたはSageMakerコンソールから数回クリックするだけでモデルを起動し、安全でスケーラブルな環境にデプロイすることができます。
"""

docs = [
    Document(
        page_content=example_doc_1,
    )
]

#### prompt の定義

ここでは LLM に入力するプロンプトを定義していきます。プロンプトには後述する `Chain` の中で得られた情報などが変数として代入されうります。  
また、カスタマイズした変数（今回のケースだと `instruction`） として `Chain` の呼び出しごとにコントロールすることも可能です。 

In [ ]:
instruction = '以下の情報を使って質問に答えてください。'

prompt_template = """システム: 以下は、ユーザーとシステムとの会話です。システムは資料から抜粋して質問に答えます。資料にない内容は答えず「わかりません」と答えます。

{context}

{instruction}
ユーザー: """
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "instruction"]
)

#### `LLM` オブジェクトの定義 

次は LLM の呼び出しに使う `LLM` オブジェクトを定義していきます。  
SageMaker Endpoint の `LLM` ラッパーとして `SagemakerEndpoint` が LangChain では用意されているためこちらを使用します。  

モデルを制御するためのパラメータ（例えばどれぐらいの長さの文章を出力するかを決める `max_new_token` など）もここで設定することになります。  
インプットするパラメータもホストするモデルによって異なるので適宜変更してください。  

In [ ]:
content_handler = ContentHandler()
llm = SagemakerEndpoint(
        endpoint_name=endpoint_name, 
        region_name=region_name, 
        model_kwargs={
            "max_new_tokens": 128,
            "temperature": 0.7,
            "do_sample": True,
            "pad_token_id": 0,
            "bos_token_id": 2,
            "eos_token_id": 265,  # 「。」の ID に相当。
            # "stop_ids": [50278, 50279, 50277, 1, 0],
        },
        content_handler=content_handler
    )

In [ ]:
chain = load_qa_chain(
    llm=llm,
    prompt=PROMPT
)

chain({"input_documents": docs, "instruction": instruction}, return_only_outputs=True)

## 後片付け

立ち上げた SageMaker Endpoint の削除を忘れないようにしましょう。  
SageMaker SDK 経由でモデルをデプロイしている場合は例えば以下のコードで実施可能です。  

```python
predictor.delete_model()
predictor.delete_endpoint()
```
